### 시각화

In [36]:
# 1. Data Load
import pandas as pd
import numpy as np

gds = pd.read_csv('../DATA/data_scaled_791226_240419.csv')
gds

,날짜,Gold,SnP,Dollar
0,1979-12-26,0.120643,0.001862,0.156976
1,1979-12-27,0.128837,0.001901,0.151836
2,1979-12-28,0.129300,0.001862,0.155049
3,1979-12-31,0.136892,0.001881,0.155156
4,1980-01-01,0.142679,0.001881,0.152479
...,...,...,...,...
11491,2024-04-15,0.985741,0.962651,0.373380
11492,2024-04-16,0.997222,0.960632,0.374880
11493,2024-04-17,0.988241,0.954969,0.370811
11494,2024-04-18,0.992686,0.952818,0.373166


In [37]:
btc = pd.read_csv('../DATA/BTCUSDT_1d.csv')
btc

,timestamp,close
0,2017-08-17,4285.08
1,2017-08-18,4108.37
2,2017-08-19,4139.98
3,2017-08-20,4086.29
4,2017-08-21,4016.00
...,...,...
2414,2024-03-27,69469.99
2415,2024-03-28,70780.60
2416,2024-03-29,69850.54
2417,2024-03-30,69582.18


In [38]:
btc['date'] = btc['timestamp'].astype('datetime64[ns]')
btc = btc.drop(['timestamp'], axis=1)
btc

,close,date
0,4285.08,2017-08-17
1,4108.37,2017-08-18
2,4139.98,2017-08-19
3,4086.29,2017-08-20
4,4016.00,2017-08-21
...,...,...
2414,69469.99,2024-03-27
2415,70780.60,2024-03-28
2416,69850.54,2024-03-29
2417,69582.18,2024-03-30


In [39]:
# gds와 btc의 날짜를 맞추기 위해 gds에서 2017-08-17 이전 데이터 삭제
gds['date'] = gds['날짜'].astype('datetime64[ns]')
gds = gds[gds['date'] >= '2017-08-17']
gds = gds[gds['date'] <= '2024-03-31']
gds = gds.drop(['날짜'], axis=1)
gds

,Gold,SnP,Dollar,date
9740,0.480857,0.452232,0.238677,2017-08-17
9741,0.480487,0.451379,0.236642,2017-08-18
9742,0.482848,0.451922,0.233001,2017-08-21
9743,0.480209,0.456596,0.237927,2017-08-22
9744,0.481922,0.454947,0.233644,2017-08-23
...,...,...,...,...
11474,0.895375,0.990148,0.353036,2024-03-26
11475,0.906902,0.998856,0.354428,2024-03-27
11476,0.918800,1.000000,0.355498,2024-03-28
11477,0.926392,0.999486,0.355070,2024-03-29


In [40]:
# 합치기
data = pd.merge(gds, btc, on='date', how='inner')
data

,Gold,SnP,Dollar,date,close
0,0.480857,0.452232,0.238677,2017-08-17,4285.08
1,0.480487,0.451379,0.236642,2017-08-18,4108.37
2,0.482848,0.451922,0.233001,2017-08-21,4016.00
3,0.480209,0.456596,0.237927,2017-08-22,4040.00
4,0.481922,0.454947,0.233644,2017-08-23,4114.01
...,...,...,...,...,...
1734,0.895375,0.990148,0.353036,2024-03-26,69988.00
1735,0.906902,0.998856,0.354428,2024-03-27,69469.99
1736,0.918800,1.000000,0.355498,2024-03-28,70780.60
1737,0.926392,0.999486,0.355070,2024-03-29,69850.54


In [41]:
# close를 btc로 변경, 데이터 정규화 (MinMaxScaler)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scale_cols = ['close']
scaled = scaler.fit_transform(data[scale_cols])
scaled = pd.DataFrame(scaled, columns=['btc'])
data = pd.concat([data, scaled], axis=1)
data = data.drop(['close'], axis=1)
data


,Gold,SnP,Dollar,date,btc
0,0.480857,0.452232,0.238677,2017-08-17,0.015684
1,0.480487,0.451379,0.236642,2017-08-18,0.013155
2,0.482848,0.451922,0.233001,2017-08-21,0.011834
3,0.480209,0.456596,0.237927,2017-08-22,0.012177
4,0.481922,0.454947,0.233644,2017-08-23,0.013236
...,...,...,...,...,...
1734,0.895375,0.990148,0.353036,2024-03-26,0.955863
1735,0.906902,0.998856,0.354428,2024-03-27,0.948451
1736,0.918800,1.000000,0.355498,2024-03-28,0.967205
1737,0.926392,0.999486,0.355070,2024-03-29,0.953896


In [43]:
# column 순서 변경 : date, close, Gold, Dollar, SnP
data = data[['date', 'btc', 'Gold', 'Dollar', 'SnP']]
data

,date,btc,Gold,Dollar,SnP
0,2017-08-17,0.015684,0.480857,0.238677,0.452232
1,2017-08-18,0.013155,0.480487,0.236642,0.451379
2,2017-08-21,0.011834,0.482848,0.233001,0.451922
3,2017-08-22,0.012177,0.480209,0.237927,0.456596
4,2017-08-23,0.013236,0.481922,0.233644,0.454947
...,...,...,...,...,...
1734,2024-03-26,0.955863,0.895375,0.353036,0.990148
1735,2024-03-27,0.948451,0.906902,0.354428,0.998856
1736,2024-03-28,0.967205,0.918800,0.355498,1.000000
1737,2024-03-29,0.953896,0.926392,0.355070,0.999486


In [44]:
# 결측치 확인
data.isnull().sum()

date      0
btc       0
Gold      0
Dollar    0
SnP       0
dtype: int64

In [45]:
# Save data
data.to_csv('../DATA/data_with_btc.csv', index='date')

### 시각화는 다음 페이지에